## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,-12.03,97,100,8.90,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.40,100,75,10.36,moderate rain
2,2,Kamaishi,JP,39.2667,141.8833,45.86,84,100,5.66,overcast clouds
3,3,Tasiilaq,GL,65.6145,-37.6368,33.80,75,75,9.22,broken clouds
4,4,Yellowknife,CA,62.4560,-114.3525,-2.00,59,40,12.66,snow


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,73.40,100,75,10.36,moderate rain
10,10,Rikitea,PF,-23.1203,-134.9692,79.86,74,10,9.48,clear sky
11,11,Pisco,PE,-13.7000,-76.2167,81.00,53,0,13.80,clear sky
13,13,Cape Town,ZA,-33.9258,18.4232,70.00,60,0,20.71,clear sky
14,14,Hithadhoo,MV,-0.6000,73.0833,83.16,74,100,8.81,overcast clouds
20,20,Chuy,UY,-33.6971,-53.4616,72.77,73,86,14.29,overcast clouds
27,27,Omboue,GA,-1.5746,9.2618,79.77,84,81,6.40,broken clouds
28,28,Castro,BR,-24.7911,-50.0119,76.14,81,82,5.12,moderate rain
30,30,Bengkulu,ID,-3.8004,102.2655,76.57,89,100,6.53,overcast clouds
33,33,Alta Floresta,BR,-9.8756,-56.0861,84.60,76,60,1.90,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.40,moderate rain,-21.2000,-175.2000,
10,Rikitea,PF,79.86,clear sky,-23.1203,-134.9692,
11,Pisco,PE,81.00,clear sky,-13.7000,-76.2167,
13,Cape Town,ZA,70.00,clear sky,-33.9258,18.4232,
14,Hithadhoo,MV,83.16,overcast clouds,-0.6000,73.0833,
20,Chuy,UY,72.77,overcast clouds,-33.6971,-53.4616,
27,Omboue,GA,79.77,broken clouds,-1.5746,9.2618,
28,Castro,BR,76.14,moderate rain,-24.7911,-50.0119,
30,Bengkulu,ID,76.57,overcast clouds,-3.8004,102.2655,
33,Alta Floresta,BR,84.60,broken clouds,-9.8756,-56.0861,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
# clean_hotel_df.head()

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))